In [1]:
import nemo.collections.asr as nemo_asr
import os
import json
from tqdm import tqdm

fine_tuned_model = "./model/parakeet-spring-lab-asr-task-wavs-finetuned-model/parakeet_finetuned.nemo"

asr_model = nemo_asr.models.ASRModel.restore_from(restore_path=fine_tuned_model)

[NeMo I 2025-06-25 09:52:41 mixins:181] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-06-25 09:52:42 modelPT:181] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: ../data/springlab-asr-task-wavs/train_manifest.json
    sample_rate: 16000
    batch_size: 4
    shuffle: true
    trim_silence: true
    max_duration: 20.0
    min_duration: 0.1
    normalize_transcripts: true
    
[NeMo W 2025-06-25 09:52:42 modelPT:188] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: ../data/springlab-asr-task-wavs/validation_manifest.json
    sample_rate: 16000
    batch_size: 4
    shuffle: false
    normalize_transcripts: true
    


[NeMo I 2025-06-25 09:52:42 features:305] PADDING: 0
[NeMo I 2025-06-25 09:52:46 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-06-25 09:52:46 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-06-25 09:52:46 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-06-25 09:52:46 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-06-25 09:52:46 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-06-25 09:52:49 save_restore_connector:282] Model EncDecRNNTBPEModel was successfully restored from /app/src/model/parakeet-spring-lab-asr-task-wavs-finetuned/parakeet_finetuned.nemo.


### Single Audio file

In [2]:
audio_file_path = "../data/2086-149220-0033.wav"

output = asr_model.transcribe([audio_file_path])
print(output[0].text)

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.22s/it]

well i dont wish to see it any more observed pebe turning away her eyes it is certainly very like the old portrait


In [3]:
import json

with open("../data/springlab-asr-task-wavs/test_manifest.json") as f:
    for line in f:
        data = json.loads(line)
        break

In [4]:
data

{'audio_filepath': '/app/data/springlab-asr-task-wavs/test_audio_0.wav',
 'duration': 6.41,
 'text': 'systems spectral technology enables the development of new mobile applications'}

### Multiple Files 

In [1]:
import os
import json
import nemo.collections.asr as nemo_asr
from tqdm import tqdm

def load_test_manifest(manifest_path):
    """Load test manifest and return list of samples"""
    samples = []
    
    print(f"Loading test manifest from: {manifest_path}")
    
    with open(manifest_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:  # Skip empty lines
                try:
                    sample = json.loads(line)
                    samples.append(sample)
                except json.JSONDecodeError as e:
                    print(f"Error parsing line: {line}")
                    print(f"Error: {e}")
    
    print(f"Loaded {len(samples)} samples from manifest")
    return samples

def inference_with_manifest(model_path, manifest_path):
    """Inference using test manifest with ground truth comparison"""
    
    print(f"Loading fine-tuned model from: {model_path}")
    asr_model = nemo_asr.models.ASRModel.restore_from(restore_path=model_path)
    print("Model loaded successfully!")
    

    test_samples = load_test_manifest(manifest_path)
    
    results = []
    
    # Process each sample from manifest
    print("Starting inference with ground truth comparison...")
    
    for sample in tqdm(test_samples, desc="Processing test samples"):
        # Extract information from manifest
        audio_filepath = sample.get('audio_filepath', '')
        original_text = sample.get('text', '')
        duration = sample.get('duration', 0)
        
        # Get filename for reference
        filename = os.path.basename(audio_filepath)

        output = asr_model.transcribe([audio_filepath])
        predicted_text = output[0].text
        
        
        # Store result with comparison
        result = {
            "filename": filename,
            "audio_filepath": audio_filepath,
            "original_text": original_text,
            "predicted_text": predicted_text,
            "duration": duration
        }
        results.append(result)
    
    return results

In [2]:
fine_tuned_model = "./model/parakeet-spring-lab-asr-task-wavs-finetuned-model/parakeet_finetuned.nemo"

test_manifest_path = "../data/springlab-asr-task-wavs/test_manifest.json"

total_results = inference_with_manifest(model_path = model_path, manifest_path = test_manifest_path)

Loading fine-tuned model from: ./model/parakeet-tdt-0.6b-v2/parakeet-tdt-0.6b-v2.nemo
[NeMo I 2025-06-25 12:19:27 mixins:181] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-06-25 12:19:28 modelPT:181] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 2
    pin_memory: true
    max_duration: 40.0
    min_duration: 0.1
    text_field: answer
    batch_duration: null
    use_bucketing: true
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: 30
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-06-25 12:19:28 modelPT:188] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    use_

[NeMo I 2025-06-25 12:19:28 features:305] PADDING: 0
[NeMo I 2025-06-25 12:19:33 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-06-25 12:19:33 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-06-25 12:19:33 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-06-25 12:19:33 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-06-25 12:19:33 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: No `cuda-python` module. Please do `pip install cuda-python>=12.3`


[NeMo I 2025-06-25 12:19:39 save_restore_connector:282] Model EncDecRNNTBPEModel was successfully restored from /app/src/model/parakeet-tdt-0.6b-v2/parakeet-tdt-0.6b-v2.nemo.
Model loaded successfully!
Loading test manifest from: ../data/springlab-asr-task-wavs/test_manifest.json
Loaded 800 samples from manifest
Starting inference with ground truth comparison...


Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.33it/s]

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.24it/s]

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.72it/s]

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.25it/s]

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.06it/s]

Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.05it/s]

Transcribing: 100%|█████████████████████

In [3]:
# dump into .json file

output_dir = "../data/springlab-asr-task-wavs-test-data-v0.json"

with open(output_dir, 'w', encoding='utf-8') as f:
    json.dump(total_results, f, indent=2, ensure_ascii=False)